In [ ]:
#이 셀은 실행하지 않으셔도 됩니다
#url 나누기
arr = list(arr)
text1 = arr[:threshold]
text2 = arr[threshold: 2*threshold]
text3 = arr[2*threshold: 3*threshold]
text4 = arr[3*threshold:]
texts = [text1, text2, text3, text4]

for k in range(len(texts)):
    with open('url_part' + str(k+1) + '.txt', 'w') as f:
        for url in texts[k]:
            f.write(url + '\n')

# 이 이후부터 실행!

In [10]:
#함수 정의
from selenium import webdriver
from urllib.parse import quote
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd 
import re

char = r'#[\d\w]+'
char2 = re.compile('[^ 0-9a-zA-Zㄱ-ㅣ가-힣!#?]')

#URL에서 데이터 가져오기
def IdHashTagFromInstagram(browser,url):
    browser.get(url)
    datetime = browser.find_element_by_class_name('_1o9PC.Nzb55').get_attribute('title')
    href=browser.find_elements_by_class_name('C4VMK')
    #포스트 내용(글, 태그)
    total_hash_text=[]    
    for i in range(0, len(href)):
        hash_text= href[i].find_element_by_css_selector('span').text
        total_hash_text.append(hash_text)
    tags = re.findall(char, char2.sub(' ',str(total_hash_text)))
    user_name = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[1]/div[1]/h2/a').text
    like_count = int(browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/div[2]/section[2]/div/div/button/span').text)
    location = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a').text
    loc_url = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a').get_attribute('href')
    return datetime, user_name, like_count, tags, location, url, loc_url

In [11]:
k = 1
browser = webdriver.Chrome('c:/Temp/chromedriver.exe')
insta_df=pd.DataFrame(columns = {'Datetime', 'User_Name', 'Like_Count', 'Tags', 'Location','URL', 'Location_URL'})

with open('url_part' + str(k) + '.txt', 'r') as f:
    urls = f.readlines()
for url in urls:
    try:
        datetime, user_name, like_count, tags, location, url, loc_url = IdHashTagFromInstagram(browser, url.strip('\n'))
        insta_df = insta_df.append({'Datetime': datetime, 'User_Name':user_name, 'Like_Count':like_count,'Tags': tags, 'Location' : location, 'URL':url,'Location_URL' : loc_url}, ignore_index = True)
    except:
        continue

In [14]:
keyword = '치즈케이크'

In [16]:
#csv 저장
insta_df.to_csv(keyword + str(k) + '.csv',mode='w', encoding='UTF-8')

### 지도 URL에서 Latitude, Longitude 수집 크롤링

In [18]:
handle = '아이디'
pwd = '비밀번호'
driverdir = '크롬드라이버 경로'
k = 1

In [20]:
#raw data 로드
insta_df = pd.read_csv('testtesttest' + k + '.csv', index_col=0, encoding='UTF-8')

# 로그인
browser= webdriver.Chrome(driverdir)
iurl='https://www.instagram.com/p/B6W9LyMhTjB'
browser.get(iurl)
time.sleep(3)
login = browser.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button')
login.click()
time.sleep(3)
user_id = browser.find_element_by_class_name('_2hvTZ.pexuQ.zyHYP')
pw = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[3]/div/label/input')
user_id.send_keys(handle)
pw.send_keys(pwd)
button = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button')
button.click()
time.sleep(3)

#주소URL에서 위도, 경도 가져오기
def grab_and_clean_selenium(url):
    browser.get(url)
    try:
        lat = browser.find_element_by_xpath("//meta[@property='place:location:latitude']").get_attribute("content")
        long = browser.find_element_by_xpath("//meta[@property='place:location:longitude']").get_attribute("content")
    except:
        lat = 'N/A'
        long= 'N/A'
    return lat,long

coord = pd.DataFrame(columns=['Latitude','Longitude','Location_URL'])
for url in insta_df['Location_URL']:
    lat, long = grab_and_clean_selenium(url)
    coord = coord.append({'Latitude':lat,'Longitude':long,'Location_URL':url}, ignore_index=True)

insta_df_final = pd.merge(insta_df,coord)
insta_df_final.to_csv('rawdata_final.csv',mode='w', encoding='UTF-8')

KeyboardInterrupt: 